In [2]:
from transformers import BertTokenizerFast, BertForMaskedLM, BertModel
import pandas as pd

In [3]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
# model = BertForMaskedLM.from_pretrained('bert-base-chinese')
model = BertForMaskedLM.from_pretrained('ckiplab/bert-base-chinese')

Downloading:   0%|          | 0.00/701 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/390M [00:00<?, ?B/s]

In [27]:
cxdata = pd.read_csv("../data/raw_cx_data.csv")

In [72]:
import ast
def eval_obj(x):
    ret = {}
    for k, v in x.items():
        if k == "board":
            ret[k] = v
        else:
            ret[k] = ast.literal_eval(v)
    return ret
data = cxdata.to_dict(orient="records")
data = [eval_obj(x) for x in data]

In [75]:
import json
with open("../data/raw_cx_data.json", "w", encoding="UTF-8") as fout:
    json.dump(data, fout, indent=2, ensure_ascii=False)

In [108]:
[i for i, x in enumerate(data) if len("".join(x["cnstr_example"])) != len(x["cnstr_example"])][10:20]

[669, 1211, 1214, 1221, 1222, 1223, 1233, 1325, 1378, 1384]

In [110]:
xx = data[1211]
xx

{'board': 'BabyMother',
 'text': ['再', '擠', '也', '擠', '不', '出來', '了'],
 'cnstr': ['O', 'BX', 'IX', 'IX', 'IX', 'IX', 'O'],
 'slot': ['O', 'BV', 'BC', 'BV', 'BC', 'BV', 'O'],
 'cnstr_form': ['v', '也', 'v', '不', 'X'],
 'cnstr_example': ['擠', '也', '擠', '不', '出來']}

In [131]:
cx_inst = data[1211]
def get_masked(cx_inst):
    cx_mask = [x!="O" for x in cx_inst["cnstr"]]
    cx_cslot = [x.endswith("C") for x in cx_inst["slot"]]
    cx_vslot = [x.endswith("V") for x in cx_inst["slot"]]
    return {"cx": cx_mask, "cslot": cx_cslot, "vslot": cx_vslot}

def get_masked_indices(masked):
    return [i for i, x in enumerate(masked) if x]

def get_masked_text(cx_inst, mask_field):
    text = cx_inst["text"]
    mask_dict = get_masked(cx_inst)
    MASK_FIELDS = ("cx", "cslot", "vslot")
    if mask_field not in MASK_FIELDS:
        raise ValueError("Unsupported mask fields")
    target_mask = mask_dict[mask_field]
    masked_text = [(["[MASK]"]*len(t) if m else [t]) for t, m in zip(text, target_mask)]
    masked_text = list(chain.from_iterable(masked_text))
    masked_indices = get_masked_indices(target_mask)
    
    return {"masked": masked_text, 
            "text": text, "mindex": masked_indices}

In [133]:
cx_inst

{'board': 'BabyMother',
 'text': ['再', '擠', '也', '擠', '不', '出來', '了'],
 'cnstr': ['O', 'BX', 'IX', 'IX', 'IX', 'IX', 'O'],
 'slot': ['O', 'BV', 'BC', 'BV', 'BC', 'BV', 'O'],
 'cnstr_form': ['v', '也', 'v', '不', 'X'],
 'cnstr_example': ['擠', '也', '擠', '不', '出來']}

In [132]:
get_masked_text(cx_inst, "cx")

{'masked': ['再',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '[MASK]',
  '了'],
 'text': ['再', '擠', '也', '擠', '不', '出來', '了'],
 'mindex': [1, 2, 3, 4, 5]}

In [121]:
from itertools import chain
target_mask = cx_vslot # cx_mask, cx_cslot, cx_vslot
masked_text = [(["[MASK]"]*len(t) if m else [t]) for t, m in zip(cx_inst["text"], target_mask)]
masked_text = list(chain.from_iterable(masked_text))
ori_text = cx_inst["text"]

In [122]:
print(masked_text)
print(ori_text)

['再', '[MASK]', '也', '[MASK]', '不', '[MASK]', '[MASK]', '了']
['再', '擠', '也', '擠', '不', '出來', '了']


In [124]:
X = tokenizer(masked_text, return_tensors="pt", is_split_into_words=True)
Xori = tokenizer(ori_text, return_tensors="pt", is_split_into_words=True)
out = model(**X, labels=Xori["input_ids"])
tokenizer.decode(out.logits.argmax(dim=2).squeeze())
out.loss

tensor(5.1272, grad_fn=<NllLossBackward0>)